In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
data_x=pd.read_csv("F:\Personal\Queens MMAI\MMAI 869 – Machine Learning and AI Technology\Driven Data team Assignment\Train set values.csv")
data_y=pd.read_csv("F:\Personal\Queens MMAI\MMAI 869 – Machine Learning and AI Technology\Driven Data team Assignment\Train set labels.csv")

In [33]:
data=pd.merge(data_x,data_y,on="id",how="inner")

## Splitting the data into training and testing

In [34]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data_x,data_y,train_size=0.7,random_state=42,stratify=data_y["status_group"])

## Data Exploration 

In [35]:
#Numerical Features Description
x_train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,41580.000000,41580.000000,41580.000000,41580.000000,4.158000e+04,41580.000000,41580.000000,41580.000000,41580.000000,41580.000000
mean,37053.095238,328.297768,670.883333,34.088923,-5.716324e+00,0.500000,15.198509,5.593218,180.270178,1303.064791
std,21379.253543,3380.560532,693.624472,6.541418,2.940352e+00,14.103926,17.463795,9.575067,462.724333,950.851187
min,0.000000,0.000000,-63.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18530.750000,0.000000,0.000000,33.097738,-8.543252e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37055.500000,0.000000,374.000000,34.911885,-5.036156e+00,0.000000,12.000000,3.000000,27.500000,1986.000000
75%,55351.250000,25.000000,1322.000000,37.169114,-3.328690e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.344301,-2.000000e-08,1776.000000,99.000000,80.000000,15300.000000,2013.000000


In [36]:
#Categorical Features Description
x_train.describe(include="O")

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,41580,39034,39025,41580,41580,41336,41580,41580,41580,39283,...,41580,41580,41580,41580,41580,41580,41580,41580,41580,41580
unique,346,1611,1802,27265,9,16078,21,124,2073,2,...,7,8,6,5,5,10,7,3,7,6
top,2011-03-17,Government Of Tanzania,DWE,none,Lake Victoria,Majengo,Iringa,Njombe,Igosi,True,...,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,410,6405,12204,2546,7058,364,3752,1769,222,35748,...,17792,35496,35496,23232,23232,11944,11944,32085,19911,24183


## Data Preparation

In [37]:
# making a copy of x_train and y_train. All processing will be done on the copies.

In [72]:
x_train_2=x_train.copy()
y_train_2=y_train.copy()

In [73]:
def process(a,b):
    ## new feature processing
    for i in ["funder","installer"]:
        a[i]=a[i].str.lower()
        minor_values=a[i].value_counts(dropna=False)[a[i].value_counts(dropna=False,normalize=True).cumsum().values>0.95].index
        a[i]=a[i].apply(lambda s:"minor" if s in minor_values else s)
        
    a=a.drop(["id","recorded_by","extraction_type_group","extraction_type_class","management_group","water_quality","quantity_group","source","source_class","waterpoint_type_group","num_private"],axis=1)
    b=b.drop("id",axis=1)
    a["long_mod"]=a["longitude"]
    a.loc[a['long_mod'] == 0, 'long_mod'] = pd.NA
    grp_means=a.groupby(by=["region"])["long_mod"].transform("mean")
    a.long_mod=a.long_mod.fillna(grp_means)
    a["lat_mod"]=a["latitude"]
    a.loc[a['lat_mod'] <-100, 'lat_mod'] = pd.NA
    grp_means2=a.groupby(by=["region"])["lat_mod"].transform("mean")
    a.lat_mod=a.lat_mod.fillna(grp_means2)
    a["gps_height_mod"]=a["gps_height"]
    a.loc[a['gps_height_mod'] == 0, 'gps_height_mod'] = pd.NA
    grp_means3=a.groupby(by=["basin"])["gps_height_mod"].transform("mean")
    a.gps_height_mod=a.gps_height_mod.fillna(grp_means3)
    #a["amount_tsh_cat"] = a["amount_tsh"].apply(lambda x: 0 if x == 0 else 1)
    date_format = '%Y-%m-%d'
    a[["rec_year","rec_month","rec_day"]]=a.date_recorded.str.split("-",n=2,expand=True)
    #a["date_recorded"]=pd.to_datetime(a["date_recorded"])
    #a["dsr"]=pd.to_datetime("2023-11-16")-a["date_recorded"]
    #a.dsr=(a.dsr/ pd.Timedelta(seconds=86400)).astype("O")
    #bin_edges=[np.min(a.gps_height)-1,0,500,1000,1500,2000,np.max(a.gps_height)]
    #bin_labels=["<0","0-500","500-1000","1000-1500","1500-2000",">2000"]
    #a["gps_height_cat"]=pd.cut(a.gps_height,bins=bin_edges,labels=bin_labels).astype("O")
    #a["dst"]=np.sqrt(a["latitude"]**2+a["longitude"]**2)
    #bin_edges=[np.min(a.construction_year)-1,0,1961,1971,1981,1991,2001,np.max(a.gps_height)+1]
    #bin_labels=["0","0-1960","1960-70","1970-80","1980-90","1990-2000","2001-2013"]
    #a["construction_year_cat"]=pd.cut(a.construction_year,bins=bin_edges,labels=bin_labels).astype("O")
    a["const"]=a["construction_year"]
    a.loc[a["const"] == 0, "const"] = pd.NA
    grp_mode=a.groupby(by=["basin"])["const"].transform(lambda x: x.mode().iloc[0])
    a["const"]=a["const"].fillna(grp_mode)  
    a=a.drop(["date_recorded","gps_height","latitude","longitude","construction_year"],axis=1)
    a[["region_code","district_code","const","rec_year","rec_month","rec_day"]]=a[["region_code","district_code","const","rec_year","rec_month","rec_day"]].astype("O")
    cat=a.select_dtypes(include="O").columns
    a[cat] = a[cat].fillna('missing')
    a[["region_code","district_code","const","rec_year","rec_month","rec_day"]]=a[["region_code","district_code","const","rec_year","rec_month","rec_day"]].astype("O")
    return a,b
    

In [74]:
x_train_2,y_train_2=process(x_train_2,y_train_2)

In [75]:
x_train_2.dtypes

amount_tsh           float64
funder                object
installer             object
wpt_name              object
basin                 object
subvillage            object
region                object
region_code           object
district_code         object
lga                   object
ward                  object
population             int64
public_meeting        object
scheme_management     object
scheme_name           object
permit                object
extraction_type       object
management            object
payment               object
payment_type          object
quality_group         object
quantity              object
source_type           object
waterpoint_type       object
long_mod             float64
lat_mod              float64
gps_height_mod       float64
rec_year              object
rec_month             object
rec_day               object
const                 object
dtype: object

In [76]:
x_train_2.isna().sum()

amount_tsh           0
funder               0
installer            0
wpt_name             0
basin                0
subvillage           0
region               0
region_code          0
district_code        0
lga                  0
ward                 0
population           0
public_meeting       0
scheme_management    0
scheme_name          0
permit               0
extraction_type      0
management           0
payment              0
payment_type         0
quality_group        0
quantity             0
source_type          0
waterpoint_type      0
long_mod             0
lat_mod              0
gps_height_mod       0
rec_year             0
rec_month            0
rec_day              0
const                0
dtype: int64

In [77]:
x_test_2=x_test.copy()
y_test_2=y_test.copy()
x_test_2,y_test_2=process(x_test_2,y_test_2)

In [78]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
def matrix(xtr,ytr,xte,yte,est):
    print ("\n Train accuracy",round(accuracy_score(ytr, est.predict(xtr)),3))
    print ("\n Train Classification Report\n\n", classification_report(ytr, est.predict(xtr)))
    print ("\n Test accuracy", round(accuracy_score(yte, est.predict(xte)),3))
    print ("\n Test Classification Report\n\n", classification_report(yte, est.predict(xte)))

In [79]:
from catboost import CatBoostClassifier

In [46]:
cat_col=list(x_train_2.select_dtypes(include="O").columns)
clf = CatBoostClassifier(iterations=10, depth=5, learning_rate=0.1, loss_function='MultiClass',cat_features=cat_col)

In [47]:
from sklearn.model_selection import cross_val_score

In [48]:
y_train_2

,status_group
5974,functional
31787,functional
27800,functional
8008,non functional
39725,non functional
...,...
21108,functional needs repair
14847,non functional
26579,non functional
8192,functional


In [153]:
cat_col=list(x_train_2.select_dtypes(include="O").columns)
clf = CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col)
parameters={"iterations":np.arange(10,110,10),"depth":np.arange(10,120,20),"learning_rate":[0.1]}
Grid=GridSearchCV(clf,param_grid=parameters,cv=5,scoring="accuracy")

In [154]:
Grid.fit(x_train_2,y_train_2)

0:	learn: 1.0248153	total: 212ms	remaining: 1.91s
1:	learn: 0.9669819	total: 420ms	remaining: 1.68s
2:	learn: 0.9185694	total: 620ms	remaining: 1.45s
3:	learn: 0.8767320	total: 827ms	remaining: 1.24s
4:	learn: 0.8420713	total: 1.03s	remaining: 1.03s
5:	learn: 0.8126578	total: 1.21s	remaining: 809ms
6:	learn: 0.7874836	total: 1.42s	remaining: 606ms
7:	learn: 0.7636150	total: 1.64s	remaining: 411ms
8:	learn: 0.7420164	total: 1.84s	remaining: 205ms
9:	learn: 0.7238264	total: 2.06s	remaining: 0us
0:	learn: 1.0247878	total: 202ms	remaining: 1.82s
1:	learn: 0.9655781	total: 393ms	remaining: 1.57s
2:	learn: 0.9167279	total: 583ms	remaining: 1.36s
3:	learn: 0.8750651	total: 771ms	remaining: 1.16s
4:	learn: 0.8404164	total: 956ms	remaining: 956ms
5:	learn: 0.8112060	total: 1.14s	remaining: 761ms
6:	learn: 0.7857892	total: 1.33s	remaining: 570ms
7:	learn: 0.7638486	total: 1.51s	remaining: 379ms
8:	learn: 0.7429936	total: 1.71s	remaining: 190ms
9:	learn: 0.7255384	total: 1.94s	remaining: 0us
0:	l

6:	learn: 0.7842468	total: 1.38s	remaining: 2.56s
7:	learn: 0.7604609	total: 1.56s	remaining: 2.35s
8:	learn: 0.7418175	total: 1.67s	remaining: 2.04s
9:	learn: 0.7241798	total: 1.87s	remaining: 1.87s
10:	learn: 0.7081962	total: 2.05s	remaining: 1.68s
11:	learn: 0.6948862	total: 2.24s	remaining: 1.49s
12:	learn: 0.6825041	total: 2.43s	remaining: 1.31s
13:	learn: 0.6717361	total: 2.62s	remaining: 1.12s
14:	learn: 0.6618280	total: 2.82s	remaining: 939ms
15:	learn: 0.6511959	total: 3s	remaining: 751ms
16:	learn: 0.6427322	total: 3.21s	remaining: 567ms
17:	learn: 0.6354652	total: 3.42s	remaining: 381ms
18:	learn: 0.6288555	total: 3.62s	remaining: 190ms
19:	learn: 0.6219958	total: 3.83s	remaining: 0us
0:	learn: 1.0243612	total: 202ms	remaining: 3.84s
1:	learn: 0.9653196	total: 409ms	remaining: 3.68s
2:	learn: 0.9162810	total: 607ms	remaining: 3.44s
3:	learn: 0.8747885	total: 800ms	remaining: 3.2s
4:	learn: 0.8391424	total: 1s	remaining: 3s
5:	learn: 0.8098040	total: 1.19s	remaining: 2.77s
6:

29:	learn: 0.5820941	total: 6s	remaining: 0us
0:	learn: 1.0247878	total: 200ms	remaining: 5.79s
1:	learn: 0.9655781	total: 405ms	remaining: 5.67s
2:	learn: 0.9167279	total: 642ms	remaining: 5.78s
3:	learn: 0.8750651	total: 835ms	remaining: 5.42s
4:	learn: 0.8404164	total: 1.03s	remaining: 5.17s
5:	learn: 0.8112060	total: 1.23s	remaining: 4.94s
6:	learn: 0.7857892	total: 1.44s	remaining: 4.72s
7:	learn: 0.7638486	total: 1.64s	remaining: 4.5s
8:	learn: 0.7429936	total: 1.84s	remaining: 4.28s
9:	learn: 0.7255384	total: 2.03s	remaining: 4.06s
10:	learn: 0.7097105	total: 2.23s	remaining: 3.85s
11:	learn: 0.6955509	total: 2.44s	remaining: 3.66s
12:	learn: 0.6849650	total: 2.64s	remaining: 3.45s
13:	learn: 0.6735093	total: 2.84s	remaining: 3.24s
14:	learn: 0.6634603	total: 3.03s	remaining: 3.03s
15:	learn: 0.6528971	total: 3.23s	remaining: 2.82s
16:	learn: 0.6447852	total: 3.42s	remaining: 2.61s
17:	learn: 0.6375906	total: 3.62s	remaining: 2.41s
18:	learn: 0.6305907	total: 3.82s	remaining: 2.

12:	learn: 0.6821633	total: 2.48s	remaining: 3.25s
13:	learn: 0.6710418	total: 2.68s	remaining: 3.06s
14:	learn: 0.6609993	total: 2.88s	remaining: 2.88s
15:	learn: 0.6533344	total: 3.07s	remaining: 2.68s
16:	learn: 0.6451859	total: 3.27s	remaining: 2.5s
17:	learn: 0.6379922	total: 3.48s	remaining: 2.32s
18:	learn: 0.6320045	total: 3.68s	remaining: 2.13s
19:	learn: 0.6253564	total: 3.88s	remaining: 1.94s
20:	learn: 0.6183322	total: 4.08s	remaining: 1.75s
21:	learn: 0.6129823	total: 4.28s	remaining: 1.55s
22:	learn: 0.6069662	total: 4.47s	remaining: 1.36s
23:	learn: 0.6028135	total: 4.67s	remaining: 1.17s
24:	learn: 0.5986444	total: 4.86s	remaining: 972ms
25:	learn: 0.5946332	total: 5.05s	remaining: 777ms
26:	learn: 0.5903212	total: 5.25s	remaining: 583ms
27:	learn: 0.5864888	total: 5.45s	remaining: 389ms
28:	learn: 0.5827389	total: 5.65s	remaining: 195ms
29:	learn: 0.5800981	total: 5.84s	remaining: 0us
0:	learn: 1.0223011	total: 207ms	remaining: 6.01s
1:	learn: 0.9622286	total: 409ms	re

16:	learn: 0.6447852	total: 3.34s	remaining: 4.52s
17:	learn: 0.6375906	total: 3.54s	remaining: 4.33s
18:	learn: 0.6305907	total: 3.74s	remaining: 4.13s
19:	learn: 0.6240923	total: 3.95s	remaining: 3.95s
20:	learn: 0.6180444	total: 4.14s	remaining: 3.75s
21:	learn: 0.6127301	total: 4.34s	remaining: 3.55s
22:	learn: 0.6073902	total: 4.53s	remaining: 3.35s
23:	learn: 0.6035623	total: 4.72s	remaining: 3.15s
24:	learn: 0.5995346	total: 4.92s	remaining: 2.95s
25:	learn: 0.5944531	total: 5.12s	remaining: 2.76s
26:	learn: 0.5913329	total: 5.32s	remaining: 2.56s
27:	learn: 0.5876758	total: 5.53s	remaining: 2.37s
28:	learn: 0.5846884	total: 5.73s	remaining: 2.17s
29:	learn: 0.5810587	total: 5.92s	remaining: 1.97s
30:	learn: 0.5776842	total: 6.13s	remaining: 1.78s
31:	learn: 0.5740788	total: 6.33s	remaining: 1.58s
32:	learn: 0.5710644	total: 6.53s	remaining: 1.38s
33:	learn: 0.5679820	total: 6.73s	remaining: 1.19s
34:	learn: 0.5650157	total: 6.93s	remaining: 990ms
35:	learn: 0.5625172	total: 7.1

...

18:	learn: 0.6292249	total: 3.66s	remaining: 15.6s
19:	learn: 0.6229493	total: 3.85s	remaining: 15.4s
20:	learn: 0.6178411	total: 4.04s	remaining: 15.2s
21:	learn: 0.6130577	total: 4.24s	remaining: 15s
22:	learn: 0.6083633	total: 4.45s	remaining: 14.9s
23:	learn: 0.6040271	total: 4.64s	remaining: 14.7s
24:	learn: 0.5992274	total: 4.83s	remaining: 14.5s
25:	learn: 0.5947075	total: 5.03s	remaining: 14.3s
26:	learn: 0.5910118	total: 5.24s	remaining: 14.2s
27:	learn: 0.5877140	total: 5.45s	remaining: 14s
28:	learn: 0.5845904	total: 5.63s	remaining: 13.8s
29:	learn: 0.5811757	total: 5.84s	remaining: 13.6s
30:	learn: 0.5775692	total: 6.03s	remaining: 13.4s
31:	learn: 0.5736120	total: 6.22s	remaining: 13.2s
32:	learn: 0.5704567	total: 6.42s	remaining: 13s
33:	learn: 0.5676319	total: 6.62s	remaining: 12.8s
34:	learn: 0.5643933	total: 6.82s	remaining: 12.7s
35:	learn: 0.5616173	total: 7.02s	remaining: 12.5s
36:	learn: 0.5593793	total: 7.21s	remaining: 12.3s
37:	learn: 0.5572323	total: 7.41s	rem

80:	learn: 0.4959336	total: 17.3s	remaining: 4.06s
81:	learn: 0.4947221	total: 17.5s	remaining: 3.85s
82:	learn: 0.4932059	total: 17.7s	remaining: 3.63s
83:	learn: 0.4922868	total: 17.9s	remaining: 3.41s
84:	learn: 0.4917852	total: 18.1s	remaining: 3.2s
85:	learn: 0.4899729	total: 18.3s	remaining: 2.98s
86:	learn: 0.4885809	total: 18.5s	remaining: 2.76s
87:	learn: 0.4874297	total: 18.7s	remaining: 2.55s
88:	learn: 0.4864113	total: 18.9s	remaining: 2.33s
89:	learn: 0.4855636	total: 19.1s	remaining: 2.12s
90:	learn: 0.4848827	total: 19.3s	remaining: 1.91s
91:	learn: 0.4825683	total: 19.5s	remaining: 1.7s
92:	learn: 0.4814205	total: 19.7s	remaining: 1.48s
93:	learn: 0.4802600	total: 19.9s	remaining: 1.27s
94:	learn: 0.4791976	total: 20.1s	remaining: 1.06s
95:	learn: 0.4784882	total: 20.3s	remaining: 847ms
96:	learn: 0.4776580	total: 20.5s	remaining: 635ms
97:	learn: 0.4768984	total: 20.7s	remaining: 423ms
98:	learn: 0.4761457	total: 20.9s	remaining: 211ms
99:	learn: 0.4756813	total: 21.1s

C:\Users\siddh\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
500 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
500 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\siddh\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\siddh\anaconda3\Lib\site-packages\catboost\core.py", line 5131, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "C:\Users\siddh\anaconda3\Lib\site-packages\catboost\core.py", line 2341

0:	learn: 1.0240479	total: 235ms	remaining: 23.3s
1:	learn: 0.9646023	total: 446ms	remaining: 21.9s
2:	learn: 0.9159819	total: 649ms	remaining: 21s
3:	learn: 0.8748921	total: 846ms	remaining: 20.3s
4:	learn: 0.8392158	total: 1.05s	remaining: 20s
5:	learn: 0.8097586	total: 1.25s	remaining: 19.6s
6:	learn: 0.7850181	total: 1.45s	remaining: 19.2s
7:	learn: 0.7604636	total: 1.64s	remaining: 18.9s
8:	learn: 0.7396203	total: 1.84s	remaining: 18.6s
9:	learn: 0.7241478	total: 1.93s	remaining: 17.4s
10:	learn: 0.7073687	total: 2.13s	remaining: 17.3s
11:	learn: 0.6930091	total: 2.33s	remaining: 17.1s
12:	learn: 0.6804533	total: 2.53s	remaining: 16.9s
13:	learn: 0.6693012	total: 2.73s	remaining: 16.8s
14:	learn: 0.6591169	total: 2.93s	remaining: 16.6s
15:	learn: 0.6502731	total: 3.12s	remaining: 16.4s
16:	learn: 0.6420922	total: 3.32s	remaining: 16.2s
17:	learn: 0.6349124	total: 3.51s	remaining: 16s
18:	learn: 0.6278605	total: 3.71s	remaining: 15.8s
19:	learn: 0.6215435	total: 3.9s	remaining: 15.

GridSearchCV(cv=10,
             estimator=<catboost.core.CatBoostClassifier object at 0x00000239C66E1B10>,
             param_grid={'depth': array([ 10,  30,  50,  70,  90, 110]),
                         'iterations': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100]),
                         'learning_rate': [0.1]},
             scoring='f1_weighted')

In [157]:
Grid.best_params_

{'depth': 10, 'iterations': 100, 'learning_rate': 0.1}

In [156]:
Grid.best_estimator_

In [158]:
matrix(x_train_2,y_train_2,x_test_2,y_test_2,Grid)


 Train accuracy 0.829

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.80      0.94      0.87     22535
functional needs repair       0.75      0.28      0.41      3024
         non functional       0.89      0.77      0.83     16021

               accuracy                           0.83     41580
              macro avg       0.81      0.67      0.70     41580
           weighted avg       0.83      0.83      0.82     41580


 Test accuracy 0.793

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.77      0.92      0.84      9724
functional needs repair       0.67      0.22      0.33      1293
         non functional       0.84      0.73      0.78      6803

               accuracy                           0.79     17820
              macro avg       0.76      0.62      0.65     17820
           weighted avg       0.79      0.

In [1]:
Grid.best_estimator_

NameError: name 'Grid' is not defined

In [97]:
Target_x=pd.read_csv("F:\Personal\Queens MMAI\MMAI 869 – Machine Learning and AI Technology\Driven Data team Assignment\Test set values.csv")

In [98]:
Target_x

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,2011-02-24,Danida,34,Da,38.852669,-6.582841,Kwambwezi,0,...,never pay,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
14846,18990,1000.0,2011-03-21,Hiap,0,HIAP,37.451633,-5.350428,Bonde La Mkondoa,0,...,annually,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
14847,28749,0.0,2013-03-04,NaN,1476,NaN,34.739804,-4.585587,Bwawani,0,...,never pay,soft,good,insufficient,insufficient,dam,dam,surface,communal standpipe,communal standpipe
14848,33492,0.0,2013-02-18,Germany,998,DWE,35.432732,-10.584159,Kwa John,0,...,never pay,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe


In [99]:
Target1=Target_x.copy()

In [100]:
def process_target(a):
    ## new feature processing
    for i in ["funder","installer"]:
        a[i]=a[i].str.lower()
        minor_values=a[i].value_counts(dropna=False)[a[i].value_counts(dropna=False,normalize=True).cumsum().values>0.95].index
        a[i]=a[i].apply(lambda s:"minor" if s in minor_values else s)
        
    a=a.drop(["id","recorded_by","extraction_type_group","extraction_type_class","management_group","water_quality","quantity_group","source","source_class","waterpoint_type_group","num_private"],axis=1)
    a["long_mod"]=a["longitude"]
    a.loc[a['long_mod'] == 0, 'long_mod'] = pd.NA
    grp_means=a.groupby(by=["region"])["long_mod"].transform("mean")
    a.long_mod=a.long_mod.fillna(grp_means)
    a["lat_mod"]=a["latitude"]
    a.loc[a['lat_mod'] <-100, 'lat_mod'] = pd.NA
    grp_means2=a.groupby(by=["region"])["lat_mod"].transform("mean")
    a.lat_mod=a.lat_mod.fillna(grp_means2)
    a["gps_height_mod"]=a["gps_height"]
    a.loc[a['gps_height_mod'] == 0, 'gps_height_mod'] = pd.NA
    grp_means3=a.groupby(by=["basin"])["gps_height_mod"].transform("mean")
    a.gps_height_mod=a.gps_height_mod.fillna(grp_means3)
    #a["amount_tsh_cat"] = a["amount_tsh"].apply(lambda x: 0 if x == 0 else 1)
    date_format = '%Y-%m-%d'
    a[["rec_year","rec_month","rec_day"]]=a.date_recorded.str.split("-",n=2,expand=True)
    #a["date_recorded"]=pd.to_datetime(a["date_recorded"])
    #a["dsr"]=pd.to_datetime("2023-11-16")-a["date_recorded"]
    #a.dsr=(a.dsr/ pd.Timedelta(seconds=86400)).astype("O")
    #bin_edges=[np.min(a.gps_height)-1,0,500,1000,1500,2000,np.max(a.gps_height)]
    #bin_labels=["<0","0-500","500-1000","1000-1500","1500-2000",">2000"]
    #a["gps_height_cat"]=pd.cut(a.gps_height,bins=bin_edges,labels=bin_labels).astype("O")
    #a["dst"]=np.sqrt(a["latitude"]**2+a["longitude"]**2)
    #bin_edges=[np.min(a.construction_year)-1,0,1961,1971,1981,1991,2001,np.max(a.gps_height)+1]
    #bin_labels=["0","0-1960","1960-70","1970-80","1980-90","1990-2000","2001-2013"]
    #a["construction_year_cat"]=pd.cut(a.construction_year,bins=bin_edges,labels=bin_labels).astype("O")
    a["const"]=a["construction_year"]
    a.loc[a["const"] == 0, "const"] = pd.NA
    grp_mode=a.groupby(by=["basin"])["const"].transform(lambda x: x.mode().iloc[0])
    a["const"]=a["const"].fillna(grp_mode)  
    a=a.drop(["date_recorded","gps_height","latitude","longitude","construction_year"],axis=1)
    a[["region_code","district_code","const","rec_year","rec_month","rec_day"]]=a[["region_code","district_code","const","rec_year","rec_month","rec_day"]].astype("O")
    cat=a.select_dtypes(include="O").columns
    a[cat] = a[cat].fillna('missing')
    a[["region_code","district_code","const","rec_year","rec_month","rec_day"]]=a[["region_code","district_code","const","rec_year","rec_month","rec_day"]].astype("O")
    return a

In [101]:
Target1=process_target(Target1)

In [102]:
Target1[cat_col]=Target1[cat_col].astype(str)

In [80]:
x_train_2[cat_col] = x_train_2[cat_col].astype(str)
x_test_2[cat_col] = x_test_2[cat_col].astype(str)

In [104]:
Cat_8127 = CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col,max_depth=10,iterations=450,learning_rate= 0.1,l2_leaf_reg=5)
Cat_8127.fit(x_train_2,y_train_2)

0:	learn: 1.0191279	total: 813ms	remaining: 6m 5s
1:	learn: 0.9569450	total: 1.64s	remaining: 6m 7s
2:	learn: 0.9046366	total: 2.41s	remaining: 5m 58s
3:	learn: 0.8617764	total: 3.33s	remaining: 6m 11s
4:	learn: 0.8255837	total: 4.27s	remaining: 6m 20s
5:	learn: 0.7954134	total: 5.14s	remaining: 6m 20s
6:	learn: 0.7693809	total: 5.87s	remaining: 6m 11s
7:	learn: 0.7453319	total: 6.65s	remaining: 6m 7s
8:	learn: 0.7259142	total: 7.13s	remaining: 5m 49s
9:	learn: 0.7078019	total: 8.11s	remaining: 5m 56s
10:	learn: 0.6923305	total: 8.87s	remaining: 5m 53s
11:	learn: 0.6790250	total: 9.73s	remaining: 5m 55s
12:	learn: 0.6655537	total: 10.6s	remaining: 5m 57s
13:	learn: 0.6540383	total: 11.5s	remaining: 5m 56s
14:	learn: 0.6435949	total: 12.1s	remaining: 5m 51s
15:	learn: 0.6336614	total: 12.9s	remaining: 5m 50s
16:	learn: 0.6254244	total: 13.7s	remaining: 5m 48s
17:	learn: 0.6165092	total: 14.5s	remaining: 5m 47s
18:	learn: 0.6090890	total: 15.2s	remaining: 5m 44s
19:	learn: 0.6024826	tota

157:	learn: 0.4356523	total: 2m 25s	remaining: 4m 28s
158:	learn: 0.4353684	total: 2m 26s	remaining: 4m 27s
159:	learn: 0.4350051	total: 2m 26s	remaining: 4m 26s
160:	learn: 0.4346900	total: 2m 27s	remaining: 4m 25s
161:	learn: 0.4339848	total: 2m 28s	remaining: 4m 24s
162:	learn: 0.4336078	total: 2m 29s	remaining: 4m 23s
163:	learn: 0.4333296	total: 2m 30s	remaining: 4m 22s
164:	learn: 0.4328424	total: 2m 31s	remaining: 4m 21s
165:	learn: 0.4324936	total: 2m 33s	remaining: 4m 21s
166:	learn: 0.4320538	total: 2m 34s	remaining: 4m 21s
167:	learn: 0.4319026	total: 2m 35s	remaining: 4m 20s
168:	learn: 0.4314051	total: 2m 36s	remaining: 4m 19s
169:	learn: 0.4312272	total: 2m 37s	remaining: 4m 18s
170:	learn: 0.4305761	total: 2m 37s	remaining: 4m 17s
171:	learn: 0.4304059	total: 2m 38s	remaining: 4m 16s
172:	learn: 0.4302016	total: 2m 39s	remaining: 4m 15s
173:	learn: 0.4299408	total: 2m 40s	remaining: 4m 15s
174:	learn: 0.4296082	total: 2m 41s	remaining: 4m 13s
175:	learn: 0.4293238	total:

310:	learn: 0.3783280	total: 4m 51s	remaining: 2m 10s
311:	learn: 0.3781718	total: 4m 52s	remaining: 2m 9s
312:	learn: 0.3777498	total: 4m 52s	remaining: 2m 8s
313:	learn: 0.3773801	total: 4m 53s	remaining: 2m 7s
314:	learn: 0.3772174	total: 4m 54s	remaining: 2m 6s
315:	learn: 0.3767156	total: 4m 55s	remaining: 2m 5s
316:	learn: 0.3762286	total: 4m 56s	remaining: 2m 4s
317:	learn: 0.3758621	total: 4m 57s	remaining: 2m 3s
318:	learn: 0.3753255	total: 4m 58s	remaining: 2m 2s
319:	learn: 0.3749741	total: 4m 59s	remaining: 2m 1s
320:	learn: 0.3747002	total: 5m	remaining: 2m
321:	learn: 0.3745575	total: 5m 1s	remaining: 1m 59s
322:	learn: 0.3739782	total: 5m 2s	remaining: 1m 58s
323:	learn: 0.3736545	total: 5m 3s	remaining: 1m 57s
324:	learn: 0.3732913	total: 5m 4s	remaining: 1m 56s
325:	learn: 0.3730791	total: 5m 5s	remaining: 1m 56s
326:	learn: 0.3727062	total: 5m 6s	remaining: 1m 55s
327:	learn: 0.3723118	total: 5m 7s	remaining: 1m 54s
328:	learn: 0.3721760	total: 5m 8s	remaining: 1m 53s

In [105]:
matrix(x_train_2,y_train_2,x_test_2,y_test_2,Cat_8127)


 Train accuracy 0.883

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.86      0.96      0.91     22581
functional needs repair       0.79      0.43      0.56      3022
         non functional       0.93      0.86      0.89     15977

               accuracy                           0.88     41580
              macro avg       0.86      0.75      0.79     41580
           weighted avg       0.88      0.88      0.88     41580


 Test accuracy 0.812

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.80      0.91      0.85      9678
functional needs repair       0.61      0.29      0.39      1295
         non functional       0.85      0.77      0.81      6847

               accuracy                           0.81     17820
              macro avg       0.75      0.66      0.69     17820
           weighted avg       0.81      0.

In [86]:
Cat_8127.get_feature_importance(prettified=True)

,Feature Id,Importances
0,quantity,10.398046
1,ward,8.638796
2,extraction_type,7.262934
3,lga,6.660160
4,const,6.357724
5,funder,5.349412
6,waterpoint_type,4.990209
7,source_type,4.318921
8,population,4.286340
9,subvillage,3.272685


In [103]:
y_pred=Cat_8127.predict(Target1)
y_pred=pd.DataFrame(y_pred, columns=["status_group"])
y_pred["id"]=Target_x["id"]
y_pred=y_pred.reindex(["id","status_group"],axis=1)
y_pred.to_csv("F:/Personal/Queens MMAI/MMAI 869 – Machine Learning and AI Technology/Driven Data team Assignment/14.12.2028_0.813_test_cat.csv",index=False)

In [62]:
Cat_8127_2 = CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col,max_depth=12,iterations=350,learning_rate= 0.1,l2_leaf_reg=5)
Cat_8127_2.fit(x_train_2,y_train_2)

0:	learn: 1.0201821	total: 2.38s	remaining: 13m 49s
1:	learn: 0.9578181	total: 5.02s	remaining: 14m 32s
2:	learn: 0.9077455	total: 7.71s	remaining: 14m 52s
3:	learn: 0.8641611	total: 10.3s	remaining: 14m 47s
4:	learn: 0.8267155	total: 12.3s	remaining: 14m 8s
5:	learn: 0.7954015	total: 14.6s	remaining: 13m 57s
6:	learn: 0.7679564	total: 17.3s	remaining: 14m 5s
7:	learn: 0.7446943	total: 19.6s	remaining: 13m 58s
8:	learn: 0.7247210	total: 22.3s	remaining: 14m 3s
9:	learn: 0.7056780	total: 24.5s	remaining: 13m 53s
10:	learn: 0.6887329	total: 26.9s	remaining: 13m 47s
11:	learn: 0.6740151	total: 29.2s	remaining: 13m 42s
12:	learn: 0.6607968	total: 31.7s	remaining: 13m 42s
13:	learn: 0.6484207	total: 34.1s	remaining: 13m 37s
14:	learn: 0.6380467	total: 36.3s	remaining: 13m 31s
15:	learn: 0.6283007	total: 38.4s	remaining: 13m 22s
16:	learn: 0.6197731	total: 40.9s	remaining: 13m 20s
17:	learn: 0.6111583	total: 43s	remaining: 13m 12s
18:	learn: 0.6033861	total: 45.2s	remaining: 13m 8s
19:	learn

154:	learn: 0.3725644	total: 6m 39s	remaining: 8m 22s
155:	learn: 0.3712298	total: 6m 41s	remaining: 8m 19s
156:	learn: 0.3708752	total: 6m 43s	remaining: 8m 16s
157:	learn: 0.3702181	total: 6m 46s	remaining: 8m 13s
158:	learn: 0.3698733	total: 6m 48s	remaining: 8m 10s
159:	learn: 0.3691818	total: 6m 51s	remaining: 8m 8s
160:	learn: 0.3688045	total: 6m 53s	remaining: 8m 5s
161:	learn: 0.3684922	total: 6m 55s	remaining: 8m 2s
162:	learn: 0.3674979	total: 6m 58s	remaining: 8m
163:	learn: 0.3664804	total: 7m 1s	remaining: 7m 57s
164:	learn: 0.3655849	total: 7m 3s	remaining: 7m 55s
165:	learn: 0.3645984	total: 7m 6s	remaining: 7m 52s
166:	learn: 0.3642064	total: 7m 8s	remaining: 7m 49s
167:	learn: 0.3635204	total: 7m 10s	remaining: 7m 46s
168:	learn: 0.3629375	total: 7m 13s	remaining: 7m 44s
169:	learn: 0.3624850	total: 7m 16s	remaining: 7m 41s
170:	learn: 0.3619846	total: 7m 18s	remaining: 7m 39s
171:	learn: 0.3612027	total: 7m 20s	remaining: 7m 36s
172:	learn: 0.3603831	total: 7m 23s	rem

306:	learn: 0.2858462	total: 12m 50s	remaining: 1m 47s
307:	learn: 0.2854181	total: 12m 52s	remaining: 1m 45s
308:	learn: 0.2848133	total: 12m 55s	remaining: 1m 42s
309:	learn: 0.2841003	total: 12m 57s	remaining: 1m 40s
310:	learn: 0.2834496	total: 13m	remaining: 1m 37s
311:	learn: 0.2826924	total: 13m 2s	remaining: 1m 35s
312:	learn: 0.2821389	total: 13m 5s	remaining: 1m 32s
313:	learn: 0.2817760	total: 13m 7s	remaining: 1m 30s
314:	learn: 0.2815130	total: 13m 9s	remaining: 1m 27s
315:	learn: 0.2810939	total: 13m 12s	remaining: 1m 25s
316:	learn: 0.2805846	total: 13m 14s	remaining: 1m 22s
317:	learn: 0.2804172	total: 13m 17s	remaining: 1m 20s
318:	learn: 0.2802551	total: 13m 19s	remaining: 1m 17s
319:	learn: 0.2799113	total: 13m 21s	remaining: 1m 15s
320:	learn: 0.2797814	total: 13m 24s	remaining: 1m 12s
321:	learn: 0.2795286	total: 13m 26s	remaining: 1m 10s
322:	learn: 0.2786612	total: 13m 28s	remaining: 1m 7s
323:	learn: 0.2782472	total: 13m 31s	remaining: 1m 5s
324:	learn: 0.277788

In [63]:
matrix(x_train_2,y_train_2,x_test_2,y_test_2,Cat_8127_2)


 Train accuracy 0.88

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.86      0.96      0.90     22581
functional needs repair       0.79      0.42      0.55      3022
         non functional       0.93      0.86      0.89     15977

               accuracy                           0.88     41580
              macro avg       0.86      0.74      0.78     41580
           weighted avg       0.88      0.88      0.87     41580


 Test accuracy 0.813

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.80      0.91      0.85      9678
functional needs repair       0.62      0.30      0.41      1295
         non functional       0.86      0.77      0.81      6847

               accuracy                           0.81     17820
              macro avg       0.76      0.66      0.69     17820
           weighted avg       0.81      0.8

In [ ]:
Cat_8127_2.get_feature_importance(prettified=True)

In [71]:
#Target2=Target_x
#Target2=process_target2(Target2)
y_pred2=Cat_8127_2.predict(Target1)
y_pred2=pd.DataFrame(y_pred2, columns=["status_group"])
y_pred2["id"]=Target_x["id"]
y_pred2=y_pred2.reindex(["id","status_group"],axis=1)
y_pred2.to_csv("F:/Personal/Queens MMAI/MMAI 869 – Machine Learning and AI Technology/Driven Data team Assignment/Cat_0.813_test_acc.csv",index=False)

In [202]:
from sklearn.utils.class_weight import compute_class_weight
class_labels = np.unique(y_train_3)
class_weights = compute_class_weight(class_weight="balanced",classes=class_labels, y=y_train_3.values.reshape(-1))
class_weights_dict = {class_label: weight for class_label, weight in zip(class_labels, class_weights)}
class_weights_dict

{'functional': 0.615043266030619,
 'functional needs repair': 4.583333333333333,
 'non functional': 0.8651145371699644}

In [203]:
Cat_8127_3 = CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col2,max_depth=10,iterations=350,learning_rate= 0.1,l2_leaf_reg=5,class_weights=class_weights_dict)
Cat_8127_3.fit(x_train_3,y_train_3)

0:	learn: 1.0501284	total: 861ms	remaining: 5m
1:	learn: 1.0067845	total: 1.7s	remaining: 4m 55s
2:	learn: 0.9717666	total: 2.39s	remaining: 4m 36s
3:	learn: 0.9436950	total: 3.16s	remaining: 4m 33s
4:	learn: 0.9176470	total: 3.93s	remaining: 4m 30s
5:	learn: 0.8965807	total: 4.7s	remaining: 4m 29s
6:	learn: 0.8752272	total: 5.46s	remaining: 4m 27s
7:	learn: 0.8571182	total: 6.29s	remaining: 4m 28s
8:	learn: 0.8416334	total: 7.13s	remaining: 4m 30s
9:	learn: 0.8268468	total: 7.85s	remaining: 4m 26s
10:	learn: 0.8136023	total: 8.63s	remaining: 4m 25s
11:	learn: 0.8014547	total: 9.32s	remaining: 4m 22s
12:	learn: 0.7892756	total: 10.1s	remaining: 4m 21s
13:	learn: 0.7794465	total: 10.7s	remaining: 4m 17s
14:	learn: 0.7698406	total: 11.4s	remaining: 4m 14s
15:	learn: 0.7616824	total: 12.5s	remaining: 4m 21s
16:	learn: 0.7543206	total: 13.4s	remaining: 4m 22s
17:	learn: 0.7473047	total: 14.2s	remaining: 4m 21s
18:	learn: 0.7409688	total: 15s	remaining: 4m 21s
19:	learn: 0.7353289	total: 16

157:	learn: 0.4950238	total: 2m 26s	remaining: 2m 57s
158:	learn: 0.4942314	total: 2m 26s	remaining: 2m 56s
159:	learn: 0.4930225	total: 2m 27s	remaining: 2m 55s
160:	learn: 0.4915821	total: 2m 28s	remaining: 2m 54s
161:	learn: 0.4901060	total: 2m 29s	remaining: 2m 53s
162:	learn: 0.4892739	total: 2m 30s	remaining: 2m 53s
163:	learn: 0.4886168	total: 2m 31s	remaining: 2m 52s
164:	learn: 0.4879273	total: 2m 32s	remaining: 2m 51s
165:	learn: 0.4867937	total: 2m 33s	remaining: 2m 50s
166:	learn: 0.4855900	total: 2m 34s	remaining: 2m 49s
167:	learn: 0.4850635	total: 2m 36s	remaining: 2m 49s
168:	learn: 0.4843669	total: 2m 37s	remaining: 2m 48s
169:	learn: 0.4837543	total: 2m 37s	remaining: 2m 47s
170:	learn: 0.4831161	total: 2m 38s	remaining: 2m 46s
171:	learn: 0.4825732	total: 2m 39s	remaining: 2m 45s
172:	learn: 0.4818955	total: 2m 40s	remaining: 2m 44s
173:	learn: 0.4814041	total: 2m 41s	remaining: 2m 43s
174:	learn: 0.4802909	total: 2m 42s	remaining: 2m 42s
175:	learn: 0.4793180	total:

311:	learn: 0.3873114	total: 4m 46s	remaining: 34.9s
312:	learn: 0.3867403	total: 4m 47s	remaining: 34s
313:	learn: 0.3862011	total: 4m 48s	remaining: 33.1s
314:	learn: 0.3858786	total: 4m 49s	remaining: 32.2s
315:	learn: 0.3851835	total: 4m 50s	remaining: 31.3s
316:	learn: 0.3850505	total: 4m 51s	remaining: 30.3s
317:	learn: 0.3844078	total: 4m 52s	remaining: 29.4s
318:	learn: 0.3839565	total: 4m 53s	remaining: 28.5s
319:	learn: 0.3832081	total: 4m 53s	remaining: 27.6s
320:	learn: 0.3829714	total: 4m 54s	remaining: 26.6s
321:	learn: 0.3827107	total: 4m 55s	remaining: 25.7s
322:	learn: 0.3820777	total: 4m 56s	remaining: 24.8s
323:	learn: 0.3811942	total: 4m 57s	remaining: 23.9s
324:	learn: 0.3806729	total: 4m 58s	remaining: 22.9s
325:	learn: 0.3801654	total: 4m 59s	remaining: 22s
326:	learn: 0.3794933	total: 5m	remaining: 21.1s
327:	learn: 0.3790842	total: 5m	remaining: 20.2s
328:	learn: 0.3782620	total: 5m 1s	remaining: 19.3s
329:	learn: 0.3778912	total: 5m 2s	remaining: 18.4s
330:	le

In [204]:
matrix(x_train_3,y_train_3,x_test_3,y_test_3,Cat_8127_3)


 Train accuracy 0.851

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.91      0.85      0.88     22535
functional needs repair       0.46      0.86      0.60      3024
         non functional       0.92      0.85      0.88     16021

               accuracy                           0.85     41580
              macro avg       0.76      0.85      0.79     41580
           weighted avg       0.88      0.85      0.86     41580


 Test accuracy 0.768

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.84      0.78      0.81      9724
functional needs repair       0.33      0.62      0.43      1293
         non functional       0.83      0.78      0.80      6803

               accuracy                           0.77     17820
              macro avg       0.67      0.72      0.68     17820
           weighted avg       0.80      0.

In [182]:
import optuna

In [216]:
from sklearn.model_selection import cross_val_score
def objective_dt(trial, X, y):
    # Now, define all the hyperparams we want to vary, and what values they are allowed
    # to take.
    #
    # Each trial, optuna will automatically choose values for each hyperparam.
    hyper_params = {
        "loss_function":'MultiClass',
        "n_estimators":100,
        "max_depth":trial.suggest_int("max_depth",8,16,step=2),
        "learning_rate":trial.suggest_float("learning_rate",0.01,.25,step=.04),
        "max_bin":trial.suggest_int("max_bin",40,80,step=20),
        "random_state": 77,
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",3,5,step=0.2),
        "eval_metric":'AUC:type=Mu',
    }


   # Use the hyperparams that optuna has chosen for this trial to create a DecisionTreeClassifier
    clf3 = CatBoostClassifier(cat_features=cat_col,**hyper_params)

  # Run CV to see how well these hyper_params do
    cv_scores = cross_val_score(clf, X, y, cv=5, scoring="f1_macro")
    score = np.mean(cv_scores)

  # Whatever we return here tells optuna how well these parameters did
    return score

In [ ]:
study.best_params

In [ ]:
study.best_value

In [213]:
Cat_8127_4 = CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col2,max_depth=10,iterations=350,learning_rate= 0.05,max_bin=60,l2_leaf_reg=5,eval_metric='AUC:type=Mu')
Cat_8127_4.fit(x_train_3,y_train_3)

0:	total: 912ms	remaining: 5m 18s
1:	total: 1.79s	remaining: 5m 11s
2:	total: 2.63s	remaining: 5m 4s
3:	total: 3.53s	remaining: 5m 5s
4:	total: 4.32s	remaining: 4m 57s
5:	total: 5.28s	remaining: 5m 2s
6:	total: 6.09s	remaining: 4m 58s
7:	total: 6.89s	remaining: 4m 54s
8:	total: 7.74s	remaining: 4m 53s
9:	total: 8.57s	remaining: 4m 51s
10:	total: 9.51s	remaining: 4m 53s
11:	total: 10.4s	remaining: 4m 53s
12:	total: 11.3s	remaining: 4m 52s
13:	total: 12.1s	remaining: 4m 51s
14:	total: 13s	remaining: 4m 49s
15:	total: 13.9s	remaining: 4m 49s
16:	total: 14.7s	remaining: 4m 47s
17:	total: 15.6s	remaining: 4m 47s
18:	total: 16.5s	remaining: 4m 46s
19:	total: 17.4s	remaining: 4m 47s
20:	total: 18.5s	remaining: 4m 49s
21:	total: 19.3s	remaining: 4m 47s
22:	total: 20.3s	remaining: 4m 48s
23:	total: 21.1s	remaining: 4m 46s
24:	total: 22s	remaining: 4m 45s
25:	total: 22.8s	remaining: 4m 43s
26:	total: 23.6s	remaining: 4m 42s
27:	total: 24.7s	remaining: 4m 43s
28:	total: 25.8s	remaining: 4m 45s
29

230:	total: 3m 28s	remaining: 1m 47s
231:	total: 3m 29s	remaining: 1m 46s
232:	total: 3m 30s	remaining: 1m 45s
233:	total: 3m 31s	remaining: 1m 44s
234:	total: 3m 32s	remaining: 1m 43s
235:	total: 3m 33s	remaining: 1m 42s
236:	total: 3m 34s	remaining: 1m 42s
237:	total: 3m 34s	remaining: 1m 41s
238:	total: 3m 35s	remaining: 1m 40s
239:	total: 3m 36s	remaining: 1m 39s
240:	total: 3m 37s	remaining: 1m 38s
241:	total: 3m 38s	remaining: 1m 37s
242:	total: 3m 39s	remaining: 1m 36s
243:	total: 3m 40s	remaining: 1m 35s
244:	total: 3m 41s	remaining: 1m 34s
245:	total: 3m 42s	remaining: 1m 33s
246:	total: 3m 43s	remaining: 1m 33s
247:	total: 3m 44s	remaining: 1m 32s
248:	total: 3m 45s	remaining: 1m 31s
249:	total: 3m 46s	remaining: 1m 30s
250:	total: 3m 46s	remaining: 1m 29s
251:	total: 3m 47s	remaining: 1m 28s
252:	total: 3m 48s	remaining: 1m 27s
253:	total: 3m 49s	remaining: 1m 26s
254:	total: 3m 50s	remaining: 1m 25s
255:	total: 3m 51s	remaining: 1m 25s
256:	total: 3m 52s	remaining: 1m 24s
2

In [214]:
matrix(x_train_3,y_train_3,x_test_3,y_test_3,Cat_8127_4)


 Train accuracy 0.864

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.84      0.95      0.89     22535
functional needs repair       0.78      0.36      0.49      3024
         non functional       0.92      0.83      0.87     16021

               accuracy                           0.86     41580
              macro avg       0.84      0.72      0.75     41580
           weighted avg       0.86      0.86      0.86     41580


 Test accuracy 0.807

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.79      0.91      0.85      9724
functional needs repair       0.67      0.28      0.39      1293
         non functional       0.85      0.76      0.80      6803

               accuracy                           0.81     17820
              macro avg       0.77      0.65      0.68     17820
           weighted avg       0.80      0.

### Removing wpt_name

In [132]:
x_train_2.wpt_name

43069      Kwa Galusi Mwajombe
58858                Kukahabwa
41707                   Polisi
30674      Kwa Mzee Shemanyaza
20944                  Ofisini
                 ...          
54343           Kwa Mbayambaya
38158                     none
860              Kwa Omar Ally
15795                 Muungano
56422    Kwa Nicholaus Kimario
Name: wpt_name, Length: 41580, dtype: object

In [133]:
x_train_21=x_train_2.drop("wpt_name",axis=1)
x_test_21=x_test_2.drop("wpt_name",axis=1)
cat_col_2=list(x_train_21.select_dtypes(include="O").columns)
Cat_8127_2=CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col_2,max_depth=10,iterations=350,learning_rate= 0.1)
Cat_8127_2.fit(x_train_21,y_train_2)

0:	learn: 1.0156315	total: 807ms	remaining: 4m 41s
1:	learn: 0.9509342	total: 1.53s	remaining: 4m 26s
2:	learn: 0.8982040	total: 2.35s	remaining: 4m 32s
3:	learn: 0.8552241	total: 3.17s	remaining: 4m 34s
4:	learn: 0.8190580	total: 4.07s	remaining: 4m 41s
5:	learn: 0.7884421	total: 5.05s	remaining: 4m 49s
6:	learn: 0.7618802	total: 5.9s	remaining: 4m 48s
7:	learn: 0.7391643	total: 6.74s	remaining: 4m 48s
8:	learn: 0.7194236	total: 7.63s	remaining: 4m 49s
9:	learn: 0.7014431	total: 8.33s	remaining: 4m 43s
10:	learn: 0.6859840	total: 9.18s	remaining: 4m 42s
11:	learn: 0.6725818	total: 10.2s	remaining: 4m 48s
12:	learn: 0.6600126	total: 11.3s	remaining: 4m 51s
13:	learn: 0.6494944	total: 12.1s	remaining: 4m 49s
14:	learn: 0.6382599	total: 12.8s	remaining: 4m 45s
15:	learn: 0.6293648	total: 13.6s	remaining: 4m 44s
16:	learn: 0.6220871	total: 13.8s	remaining: 4m 30s
17:	learn: 0.6141088	total: 14.6s	remaining: 4m 28s
18:	learn: 0.6067742	total: 15.3s	remaining: 4m 26s
19:	learn: 0.6005407	to

156:	learn: 0.4261694	total: 2m 41s	remaining: 3m 18s
157:	learn: 0.4250265	total: 2m 42s	remaining: 3m 17s
158:	learn: 0.4242596	total: 2m 43s	remaining: 3m 16s
159:	learn: 0.4238019	total: 2m 44s	remaining: 3m 15s
160:	learn: 0.4232932	total: 2m 45s	remaining: 3m 14s
161:	learn: 0.4227799	total: 2m 46s	remaining: 3m 13s
162:	learn: 0.4223265	total: 2m 47s	remaining: 3m 12s
163:	learn: 0.4220610	total: 2m 48s	remaining: 3m 11s
164:	learn: 0.4215567	total: 2m 49s	remaining: 3m 10s
165:	learn: 0.4214092	total: 2m 50s	remaining: 3m 9s
166:	learn: 0.4210094	total: 2m 51s	remaining: 3m 8s
167:	learn: 0.4205768	total: 2m 52s	remaining: 3m 7s
168:	learn: 0.4204784	total: 2m 53s	remaining: 3m 5s
169:	learn: 0.4195662	total: 2m 54s	remaining: 3m 4s
170:	learn: 0.4193365	total: 2m 55s	remaining: 3m 3s
171:	learn: 0.4185589	total: 2m 56s	remaining: 3m 3s
172:	learn: 0.4180034	total: 2m 57s	remaining: 3m 2s
173:	learn: 0.4176732	total: 2m 59s	remaining: 3m 1s
174:	learn: 0.4174103	total: 3m	remai

310:	learn: 0.3580910	total: 5m 22s	remaining: 40.5s
311:	learn: 0.3580040	total: 5m 23s	remaining: 39.4s
312:	learn: 0.3576041	total: 5m 24s	remaining: 38.4s
313:	learn: 0.3572626	total: 5m 25s	remaining: 37.3s
314:	learn: 0.3568803	total: 5m 26s	remaining: 36.3s
315:	learn: 0.3565344	total: 5m 27s	remaining: 35.2s
316:	learn: 0.3556779	total: 5m 28s	remaining: 34.2s
317:	learn: 0.3555615	total: 5m 29s	remaining: 33.1s
318:	learn: 0.3550181	total: 5m 30s	remaining: 32.1s
319:	learn: 0.3547943	total: 5m 31s	remaining: 31.1s
320:	learn: 0.3545569	total: 5m 32s	remaining: 30s
321:	learn: 0.3540623	total: 5m 33s	remaining: 29s
322:	learn: 0.3537366	total: 5m 34s	remaining: 28s
323:	learn: 0.3535715	total: 5m 35s	remaining: 26.9s
324:	learn: 0.3533963	total: 5m 36s	remaining: 25.9s
325:	learn: 0.3524460	total: 5m 37s	remaining: 24.9s
326:	learn: 0.3521995	total: 5m 38s	remaining: 23.8s
327:	learn: 0.3520658	total: 5m 40s	remaining: 22.8s
328:	learn: 0.3519304	total: 5m 41s	remaining: 21.8s

In [134]:
matrix(x_train_21,y_train_2,x_test_21,y_test_2,Cat_8127_2)


 Train accuracy 0.872

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.85      0.95      0.90     22535
functional needs repair       0.78      0.43      0.55      3024
         non functional       0.92      0.84      0.88     16021

               accuracy                           0.87     41580
              macro avg       0.85      0.74      0.78     41580
           weighted avg       0.87      0.87      0.87     41580


 Test accuracy 0.809

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.80      0.90      0.85      9724
functional needs repair       0.64      0.31      0.42      1293
         non functional       0.84      0.77      0.80      6803

               accuracy                           0.81     17820
              macro avg       0.76      0.66      0.69     17820
           weighted avg       0.80      0.

In [184]:
x_train_22=x_train_2
x_train_22["construction_year"]=x_train["construction_year"]

In [185]:
x_train_22["construction_year"].isna().sum()

0

In [186]:
x_train_22["construction_year"].dtype

dtype('int64')

In [187]:
x_test_22=x_test_2
x_test_22["construction_year"]=x_test["construction_year"]

In [188]:
x_train_22=x_train_22.drop("construction_year_cat",axis=1)
x_test_22=x_test_22.drop("construction_year_cat",axis=1)
cat_col_3=list(x_train_22.select_dtypes(include="O").columns)
Cat_8127_3=CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col_3,max_depth=10,iterations=350,learning_rate= 0.1)

In [189]:
Cat_8127_3.fit(x_train_22,y_train_2)

0:	learn: 1.0179628	total: 982ms	remaining: 5m 42s
1:	learn: 0.9540950	total: 1.87s	remaining: 5m 24s
2:	learn: 0.9019409	total: 2.81s	remaining: 5m 24s
3:	learn: 0.8584498	total: 3.78s	remaining: 5m 26s
4:	learn: 0.8224178	total: 4.71s	remaining: 5m 24s
5:	learn: 0.7910581	total: 5.59s	remaining: 5m 20s
6:	learn: 0.7644931	total: 6.57s	remaining: 5m 21s
7:	learn: 0.7409425	total: 7.4s	remaining: 5m 16s
8:	learn: 0.7189121	total: 8.16s	remaining: 5m 9s
9:	learn: 0.7003824	total: 9.05s	remaining: 5m 7s
10:	learn: 0.6849213	total: 9.94s	remaining: 5m 6s
11:	learn: 0.6701404	total: 10.9s	remaining: 5m 5s
12:	learn: 0.6567679	total: 11.7s	remaining: 5m 3s
13:	learn: 0.6462913	total: 12.7s	remaining: 5m 4s
14:	learn: 0.6362063	total: 13.7s	remaining: 5m 6s
15:	learn: 0.6266491	total: 14.8s	remaining: 5m 8s
16:	learn: 0.6183908	total: 15.6s	remaining: 5m 4s
17:	learn: 0.6102778	total: 16.5s	remaining: 5m 3s
18:	learn: 0.6035197	total: 17.5s	remaining: 5m 4s
19:	learn: 0.5976670	total: 18.5s	

157:	learn: 0.4220558	total: 2m 50s	remaining: 3m 27s
158:	learn: 0.4216877	total: 2m 51s	remaining: 3m 26s
159:	learn: 0.4212663	total: 2m 52s	remaining: 3m 25s
160:	learn: 0.4207319	total: 2m 53s	remaining: 3m 23s
161:	learn: 0.4200374	total: 2m 54s	remaining: 3m 22s
162:	learn: 0.4191171	total: 2m 56s	remaining: 3m 22s
163:	learn: 0.4184454	total: 2m 57s	remaining: 3m 21s
164:	learn: 0.4180673	total: 2m 58s	remaining: 3m 20s
165:	learn: 0.4177389	total: 2m 59s	remaining: 3m 18s
166:	learn: 0.4174901	total: 3m	remaining: 3m 18s
167:	learn: 0.4172909	total: 3m 1s	remaining: 3m 16s
168:	learn: 0.4166956	total: 3m 2s	remaining: 3m 15s
169:	learn: 0.4161145	total: 3m 3s	remaining: 3m 14s
170:	learn: 0.4156945	total: 3m 4s	remaining: 3m 13s
171:	learn: 0.4155154	total: 3m 6s	remaining: 3m 12s
172:	learn: 0.4147124	total: 3m 7s	remaining: 3m 11s
173:	learn: 0.4142075	total: 3m 8s	remaining: 3m 10s
174:	learn: 0.4140462	total: 3m 9s	remaining: 3m 9s
175:	learn: 0.4132270	total: 3m 10s	remai

311:	learn: 0.3547747	total: 5m 42s	remaining: 41.8s
312:	learn: 0.3544891	total: 5m 44s	remaining: 40.7s
313:	learn: 0.3542834	total: 5m 45s	remaining: 39.6s
314:	learn: 0.3539072	total: 5m 46s	remaining: 38.5s
315:	learn: 0.3535661	total: 5m 47s	remaining: 37.4s
316:	learn: 0.3530604	total: 5m 48s	remaining: 36.3s
317:	learn: 0.3526049	total: 5m 49s	remaining: 35.2s
318:	learn: 0.3520533	total: 5m 50s	remaining: 34.1s
319:	learn: 0.3511463	total: 5m 52s	remaining: 33s
320:	learn: 0.3507943	total: 5m 52s	remaining: 31.9s
321:	learn: 0.3501974	total: 5m 54s	remaining: 30.8s
322:	learn: 0.3498848	total: 5m 55s	remaining: 29.7s
323:	learn: 0.3494024	total: 5m 56s	remaining: 28.6s
324:	learn: 0.3490683	total: 5m 57s	remaining: 27.5s
325:	learn: 0.3487170	total: 5m 58s	remaining: 26.4s
326:	learn: 0.3484178	total: 5m 59s	remaining: 25.3s
327:	learn: 0.3477695	total: 6m	remaining: 24.2s
328:	learn: 0.3471742	total: 6m 1s	remaining: 23.1s
329:	learn: 0.3468033	total: 6m 3s	remaining: 22s
330

In [190]:
matrix(x_train_22,y_train_2,x_test_22,y_test_2,Cat_8127_3)


 Train accuracy 0.88

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.86      0.96      0.90     22535
functional needs repair       0.79      0.45      0.57      3024
         non functional       0.93      0.85      0.89     16021

               accuracy                           0.88     41580
              macro avg       0.86      0.75      0.79     41580
           weighted avg       0.88      0.88      0.87     41580


 Test accuracy 0.811

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.80      0.91      0.85      9724
functional needs repair       0.63      0.30      0.41      1293
         non functional       0.85      0.77      0.81      6803

               accuracy                           0.81     17820
              macro avg       0.76      0.66      0.69     17820
           weighted avg       0.81      0.8

In [165]:
Target2=Target1
Target2["construction_year"]=Target_x["construction_year"]
Target2=Target2.drop("construction_year_cat",axis=1)
y_pred_6=Cat_8127_3.predict(Target2)
y_pred_6=pd.DataFrame(y_pred_6, columns=["status_group"])
y_pred_6["id"]=Target_x["id"]
y_pred_6=y_pred_5.reindex(["id","status_group"],axis=1)
y_pred_6.to_csv("F:/Personal/Queens MMAI/MMAI 869 – Machine Learning and AI Technology/Driven Data team Assignment/v5_out_cat_23.11_iter_350_rate_0.1.csv_org_cons_year",index=False)

In [192]:
x_train_23=x_train_2
x_train_23["latitude"]=x_train["latitude"]
x_train_23["longitude"]=x_train["longitude"]
x_test_23=x_test_2
x_test_23["latitude"]=x_test["latitude"]
x_test_23["longitude"]=x_test["longitude"]
x_train_23=x_train_23.drop("dst",axis=1)
x_test_23=x_test_23.drop("dst",axis=1)

In [194]:
x_test_23["latitude"].dtype

dtype('float64')

In [195]:
cat_col_4=list(x_train_23.select_dtypes(include="O").columns)
Cat_8127_4=CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col_4,max_depth=10,iterations=350,learning_rate= 0.1)

In [197]:
Cat_8127_4.fit(x_train_23,y_train_2)

0:	learn: 1.0170513	total: 934ms	remaining: 5m 25s
1:	learn: 0.9509586	total: 1.73s	remaining: 5m
2:	learn: 0.8981840	total: 2.54s	remaining: 4m 54s
3:	learn: 0.8550987	total: 3.36s	remaining: 4m 50s
4:	learn: 0.8183324	total: 4.31s	remaining: 4m 57s
5:	learn: 0.7886079	total: 5.3s	remaining: 5m 4s
6:	learn: 0.7611315	total: 6.16s	remaining: 5m 1s
7:	learn: 0.7371034	total: 7.11s	remaining: 5m 3s
8:	learn: 0.7166527	total: 8.15s	remaining: 5m 9s
9:	learn: 0.6991532	total: 9.13s	remaining: 5m 10s
10:	learn: 0.6834294	total: 10s	remaining: 5m 9s
11:	learn: 0.6682676	total: 11s	remaining: 5m 9s
12:	learn: 0.6553986	total: 11.9s	remaining: 5m 9s
13:	learn: 0.6435493	total: 12.8s	remaining: 5m 8s
14:	learn: 0.6326590	total: 13.7s	remaining: 5m 5s
15:	learn: 0.6236041	total: 14.6s	remaining: 5m 5s
16:	learn: 0.6156453	total: 15.5s	remaining: 5m 3s
17:	learn: 0.6080394	total: 16.5s	remaining: 5m 3s
18:	learn: 0.6004667	total: 17.2s	remaining: 5m
19:	learn: 0.5940637	total: 18.3s	remaining: 5m

157:	learn: 0.4220652	total: 2m 57s	remaining: 3m 36s
158:	learn: 0.4215483	total: 2m 59s	remaining: 3m 35s
159:	learn: 0.4209384	total: 3m	remaining: 3m 34s
160:	learn: 0.4201343	total: 3m 1s	remaining: 3m 32s
161:	learn: 0.4194827	total: 3m 2s	remaining: 3m 31s
162:	learn: 0.4186289	total: 3m 3s	remaining: 3m 30s
163:	learn: 0.4181414	total: 3m 4s	remaining: 3m 29s
164:	learn: 0.4176024	total: 3m 5s	remaining: 3m 28s
165:	learn: 0.4163760	total: 3m 6s	remaining: 3m 27s
166:	learn: 0.4158153	total: 3m 7s	remaining: 3m 25s
167:	learn: 0.4152330	total: 3m 9s	remaining: 3m 24s
168:	learn: 0.4150990	total: 3m 10s	remaining: 3m 23s
169:	learn: 0.4140889	total: 3m 11s	remaining: 3m 22s
170:	learn: 0.4135423	total: 3m 12s	remaining: 3m 21s
171:	learn: 0.4133483	total: 3m 13s	remaining: 3m 20s
172:	learn: 0.4127014	total: 3m 14s	remaining: 3m 19s
173:	learn: 0.4121131	total: 3m 16s	remaining: 3m 18s
174:	learn: 0.4117623	total: 3m 17s	remaining: 3m 17s
175:	learn: 0.4107003	total: 3m 18s	rema

311:	learn: 0.3539914	total: 5m 49s	remaining: 42.6s
312:	learn: 0.3537064	total: 5m 50s	remaining: 41.4s
313:	learn: 0.3532623	total: 5m 51s	remaining: 40.3s
314:	learn: 0.3530896	total: 5m 52s	remaining: 39.2s
315:	learn: 0.3529416	total: 5m 53s	remaining: 38.1s
316:	learn: 0.3524625	total: 5m 54s	remaining: 36.9s
317:	learn: 0.3522313	total: 5m 56s	remaining: 35.8s
318:	learn: 0.3518930	total: 5m 57s	remaining: 34.7s
319:	learn: 0.3515282	total: 5m 58s	remaining: 33.6s
320:	learn: 0.3510264	total: 5m 59s	remaining: 32.5s
321:	learn: 0.3505628	total: 6m	remaining: 31.4s
322:	learn: 0.3498442	total: 6m 1s	remaining: 30.2s
323:	learn: 0.3494232	total: 6m 2s	remaining: 29.1s
324:	learn: 0.3491024	total: 6m 3s	remaining: 28s
325:	learn: 0.3490060	total: 6m 4s	remaining: 26.8s
326:	learn: 0.3483023	total: 6m 5s	remaining: 25.7s
327:	learn: 0.3482431	total: 6m 6s	remaining: 24.6s
328:	learn: 0.3479776	total: 6m 7s	remaining: 23.5s
329:	learn: 0.3476592	total: 6m 8s	remaining: 22.4s
330:	le

In [198]:
matrix(x_train_23,y_train_2,x_test_23,y_test_2,Cat_8127_4)


 Train accuracy 0.882

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.86      0.96      0.91     22535
functional needs repair       0.80      0.43      0.56      3024
         non functional       0.93      0.86      0.89     16021

               accuracy                           0.88     41580
              macro avg       0.86      0.75      0.79     41580
           weighted avg       0.88      0.88      0.88     41580


 Test accuracy 0.812

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.80      0.91      0.85      9724
functional needs repair       0.64      0.30      0.41      1293
         non functional       0.85      0.77      0.81      6803

               accuracy                           0.81     17820
              macro avg       0.76      0.66      0.69     17820
           weighted avg       0.81      0.

In [199]:
Cat_8127_4.get_feature_importance(prettified=True)

,Feature Id,Importances
0,ward,9.461374
1,quantity,8.445230
2,lga,7.807842
3,extraction_type,7.632415
4,funder,5.275842
5,waterpoint_type,4.897198
6,installer,4.272347
7,scheme_name,4.069568
8,source_type,3.715144
9,subvillage,3.582317


In [201]:
x_train["quantity"]

43069             dry
58858          enough
41707    insufficient
30674          enough
20944          enough
             ...     
54343          enough
38158        seasonal
860      insufficient
15795          enough
56422          enough
Name: quantity, Length: 41580, dtype: object

In [204]:
pd.crosstab(x_train["quantity"],y_train["status_group"],normalize=True)

status_group,functional,functional needs repair,non functional
quantity,,,
dry,0.002694,0.000625,0.102718
enough,0.363925,0.040741,0.153223
insufficient,0.133502,0.024050,0.097427
seasonal,0.038456,0.007071,0.022174
unknown,0.003391,0.000241,0.009764


In [206]:
Target3=Target1
Target3["latitude"]=Target_x["latitude"]
Target3["longitude"]=Target_x["longitude"]
Target3=Target3.drop("dst",axis=1)
y_pred_7=Cat_8127_4.predict(Target3)
y_pred_7=pd.DataFrame(y_pred_7, columns=["status_group"])
y_pred_7["id"]=Target_x["id"]
y_pred_7=y_pred_7.reindex(["id","status_group"],axis=1)
y_pred_7.to_csv("F:/Personal/Queens MMAI/MMAI 869 – Machine Learning and AI Technology/Driven Data team Assignment/v6_out_cat_23.11_iter_350_rate_0.1 with_long_lat.csv",index=False)

In [207]:
cat_col=list(x_train_2.select_dtypes(include="O").columns)
Cat_8127_5 = CatBoostClassifier(loss_function='MultiClassOneVsAll',cat_features=cat_col,max_depth=10,iterations=350,learning_rate= 0.1)
Cat_8127_5.fit(x_train_2,y_train_2)

0:	learn: 0.6475588	total: 1.22s	remaining: 7m 5s
1:	learn: 0.6099202	total: 2.22s	remaining: 6m 25s
2:	learn: 0.5781755	total: 3.25s	remaining: 6m 15s
3:	learn: 0.5511905	total: 4.29s	remaining: 6m 10s
4:	learn: 0.5276637	total: 5.29s	remaining: 6m 5s
5:	learn: 0.5073671	total: 6.44s	remaining: 6m 9s
6:	learn: 0.4892292	total: 7.42s	remaining: 6m 3s
7:	learn: 0.4732504	total: 8.47s	remaining: 6m 2s
8:	learn: 0.4595663	total: 9.43s	remaining: 5m 57s
9:	learn: 0.4465452	total: 10.5s	remaining: 5m 57s
10:	learn: 0.4350774	total: 11.7s	remaining: 6m 1s
11:	learn: 0.4251263	total: 12.9s	remaining: 6m 2s
12:	learn: 0.4164537	total: 14.1s	remaining: 6m 4s
13:	learn: 0.4091017	total: 15.2s	remaining: 6m 5s
14:	learn: 0.4021781	total: 16.4s	remaining: 6m 5s
15:	learn: 0.3960449	total: 17.3s	remaining: 6m
16:	learn: 0.3904259	total: 18.4s	remaining: 6m
17:	learn: 0.3848550	total: 19.4s	remaining: 5m 58s
18:	learn: 0.3800334	total: 20.5s	remaining: 5m 56s
19:	learn: 0.3758088	total: 21.7s	remain

157:	learn: 0.2772547	total: 3m 17s	remaining: 4m
158:	learn: 0.2770266	total: 3m 19s	remaining: 3m 59s
159:	learn: 0.2768683	total: 3m 20s	remaining: 3m 58s
160:	learn: 0.2763994	total: 3m 22s	remaining: 3m 57s
161:	learn: 0.2761865	total: 3m 23s	remaining: 3m 55s
162:	learn: 0.2758843	total: 3m 24s	remaining: 3m 54s
163:	learn: 0.2756479	total: 3m 25s	remaining: 3m 53s
164:	learn: 0.2754750	total: 3m 27s	remaining: 3m 52s
165:	learn: 0.2752764	total: 3m 28s	remaining: 3m 50s
166:	learn: 0.2749523	total: 3m 29s	remaining: 3m 49s
167:	learn: 0.2748216	total: 3m 31s	remaining: 3m 48s
168:	learn: 0.2746901	total: 3m 32s	remaining: 3m 47s
169:	learn: 0.2742087	total: 3m 33s	remaining: 3m 46s
170:	learn: 0.2739884	total: 3m 35s	remaining: 3m 45s
171:	learn: 0.2738570	total: 3m 36s	remaining: 3m 44s
172:	learn: 0.2736740	total: 3m 37s	remaining: 3m 42s
173:	learn: 0.2733821	total: 3m 39s	remaining: 3m 41s
174:	learn: 0.2730096	total: 3m 40s	remaining: 3m 40s
175:	learn: 0.2726984	total: 3m 

311:	learn: 0.2455971	total: 6m 32s	remaining: 47.9s
312:	learn: 0.2455066	total: 6m 34s	remaining: 46.6s
313:	learn: 0.2454079	total: 6m 35s	remaining: 45.3s
314:	learn: 0.2453282	total: 6m 36s	remaining: 44.1s
315:	learn: 0.2452079	total: 6m 37s	remaining: 42.8s
316:	learn: 0.2451341	total: 6m 39s	remaining: 41.6s
317:	learn: 0.2450368	total: 6m 40s	remaining: 40.3s
318:	learn: 0.2447389	total: 6m 41s	remaining: 39s
319:	learn: 0.2444246	total: 6m 42s	remaining: 37.8s
320:	learn: 0.2440824	total: 6m 44s	remaining: 36.5s
321:	learn: 0.2439546	total: 6m 45s	remaining: 35.2s
322:	learn: 0.2438408	total: 6m 46s	remaining: 34s
323:	learn: 0.2436284	total: 6m 47s	remaining: 32.7s
324:	learn: 0.2434753	total: 6m 48s	remaining: 31.5s
325:	learn: 0.2432564	total: 6m 50s	remaining: 30.2s
326:	learn: 0.2430691	total: 6m 51s	remaining: 28.9s
327:	learn: 0.2428864	total: 6m 52s	remaining: 27.7s
328:	learn: 0.2426820	total: 6m 53s	remaining: 26.4s
329:	learn: 0.2425254	total: 6m 54s	remaining: 25.

In [208]:
matrix(x_train_2,y_train_2,x_test_2,y_test_2,Cat_8127_5)


 Train accuracy 0.875

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.85      0.96      0.90     22535
functional needs repair       0.80      0.40      0.54      3024
         non functional       0.92      0.85      0.88     16021

               accuracy                           0.87     41580
              macro avg       0.86      0.74      0.77     41580
           weighted avg       0.88      0.87      0.87     41580


 Test accuracy 0.81

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.80      0.91      0.85      9724
functional needs repair       0.65      0.29      0.40      1293
         non functional       0.85      0.77      0.80      6803

               accuracy                           0.81     17820
              macro avg       0.76      0.66      0.69     17820
           weighted avg       0.81      0.8

In [209]:
y_pred_8=Cat_8127_5.predict(Target1)
y_pred_8=pd.DataFrame(y_pred_8, columns=["status_group"])
y_pred_8["id"]=Target_x["id"]
y_pred_8=y_pred_8.reindex(["id","status_group"],axis=1)
y_pred_8.to_csv("F:/Personal/Queens MMAI/MMAI 869 – Machine Learning and AI Technology/Driven Data team Assignment/v6_out_cat_24.11_iter_350_onevsall.csv",index=False)

In [211]:
cat_col=list(x_train_2.select_dtypes(include="O").columns)
Cat_8127_6 = CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col,max_depth=10,iterations=3500,learning_rate= 0.1,early_stopping_rounds=50)
Cat_8127_6.fit(x_train_2,y_train_2)

0:	learn: 1.0181968	total: 1.06s	remaining: 1h 2m 7s
1:	learn: 0.9523618	total: 2s	remaining: 58m 19s
2:	learn: 0.8985761	total: 2.92s	remaining: 56m 43s
3:	learn: 0.8552059	total: 4.09s	remaining: 59m 33s
4:	learn: 0.8187969	total: 5.11s	remaining: 59m 30s
5:	learn: 0.7885881	total: 6.05s	remaining: 58m 40s
6:	learn: 0.7636940	total: 7.01s	remaining: 58m 16s
7:	learn: 0.7399763	total: 8.14s	remaining: 59m 12s
8:	learn: 0.7193153	total: 9.28s	remaining: 59m 57s
9:	learn: 0.7011584	total: 10.2s	remaining: 59m 33s
10:	learn: 0.6844883	total: 11.1s	remaining: 58m 52s
11:	learn: 0.6698226	total: 12.1s	remaining: 58m 36s
12:	learn: 0.6568179	total: 13.1s	remaining: 58m 25s
13:	learn: 0.6442822	total: 14.2s	remaining: 59m 7s
14:	learn: 0.6336272	total: 15.4s	remaining: 59m 38s
15:	learn: 0.6250381	total: 16.4s	remaining: 59m 31s
16:	learn: 0.6163042	total: 17.5s	remaining: 59m 36s
17:	learn: 0.6084332	total: 18.4s	remaining: 59m 28s
18:	learn: 0.6011139	total: 19.4s	remaining: 59m 15s
19:	le

148:	learn: 0.4310944	total: 3m 10s	remaining: 1h 11m 20s
149:	learn: 0.4299641	total: 3m 12s	remaining: 1h 11m 28s
150:	learn: 0.4296374	total: 3m 13s	remaining: 1h 11m 32s
151:	learn: 0.4289084	total: 3m 14s	remaining: 1h 11m 32s
152:	learn: 0.4287117	total: 3m 16s	remaining: 1h 11m 34s
153:	learn: 0.4281161	total: 3m 17s	remaining: 1h 11m 32s
154:	learn: 0.4273615	total: 3m 18s	remaining: 1h 11m 30s
155:	learn: 0.4266139	total: 3m 20s	remaining: 1h 11m 29s
156:	learn: 0.4258914	total: 3m 21s	remaining: 1h 11m 26s
157:	learn: 0.4251096	total: 3m 22s	remaining: 1h 11m 21s
158:	learn: 0.4246359	total: 3m 23s	remaining: 1h 11m 24s
159:	learn: 0.4242619	total: 3m 25s	remaining: 1h 11m 27s
160:	learn: 0.4240305	total: 3m 26s	remaining: 1h 11m 25s
161:	learn: 0.4235028	total: 3m 27s	remaining: 1h 11m 25s
162:	learn: 0.4231691	total: 3m 29s	remaining: 1h 11m 21s
163:	learn: 0.4222621	total: 3m 30s	remaining: 1h 11m 20s
164:	learn: 0.4215577	total: 3m 31s	remaining: 1h 11m 18s
165:	learn: 0.

291:	learn: 0.3613463	total: 6m 17s	remaining: 1h 9m 9s
292:	learn: 0.3608831	total: 6m 19s	remaining: 1h 9m 10s
293:	learn: 0.3605265	total: 6m 20s	remaining: 1h 9m 9s
294:	learn: 0.3604314	total: 6m 21s	remaining: 1h 9m 8s
295:	learn: 0.3601310	total: 6m 23s	remaining: 1h 9m 8s
296:	learn: 0.3595665	total: 6m 24s	remaining: 1h 9m 11s
297:	learn: 0.3591354	total: 6m 26s	remaining: 1h 9m 12s
298:	learn: 0.3588133	total: 6m 28s	remaining: 1h 9m 15s
299:	learn: 0.3583977	total: 6m 30s	remaining: 1h 9m 21s
300:	learn: 0.3577472	total: 6m 32s	remaining: 1h 9m 29s
301:	learn: 0.3574379	total: 6m 34s	remaining: 1h 9m 38s
302:	learn: 0.3567116	total: 6m 36s	remaining: 1h 9m 41s
303:	learn: 0.3562547	total: 6m 38s	remaining: 1h 9m 45s
304:	learn: 0.3557390	total: 6m 39s	remaining: 1h 9m 50s
305:	learn: 0.3556386	total: 6m 41s	remaining: 1h 9m 52s
306:	learn: 0.3548889	total: 6m 43s	remaining: 1h 9m 54s
307:	learn: 0.3546925	total: 6m 44s	remaining: 1h 9m 54s
308:	learn: 0.3541682	total: 6m 46s

436:	learn: 0.3127701	total: 9m 32s	remaining: 1h 6m 50s
437:	learn: 0.3126329	total: 9m 33s	remaining: 1h 6m 47s
438:	learn: 0.3124590	total: 9m 34s	remaining: 1h 6m 46s
439:	learn: 0.3122369	total: 9m 36s	remaining: 1h 6m 46s
440:	learn: 0.3119760	total: 9m 37s	remaining: 1h 6m 43s
441:	learn: 0.3113511	total: 9m 38s	remaining: 1h 6m 43s
442:	learn: 0.3110983	total: 9m 39s	remaining: 1h 6m 42s
443:	learn: 0.3108426	total: 9m 41s	remaining: 1h 6m 40s
444:	learn: 0.3104349	total: 9m 42s	remaining: 1h 6m 40s
445:	learn: 0.3100878	total: 9m 44s	remaining: 1h 6m 39s
446:	learn: 0.3099176	total: 9m 45s	remaining: 1h 6m 38s
447:	learn: 0.3097875	total: 9m 46s	remaining: 1h 6m 35s
448:	learn: 0.3095654	total: 9m 47s	remaining: 1h 6m 33s
449:	learn: 0.3093009	total: 9m 49s	remaining: 1h 6m 32s
450:	learn: 0.3090360	total: 9m 50s	remaining: 1h 6m 31s
451:	learn: 0.3087705	total: 9m 51s	remaining: 1h 6m 29s
452:	learn: 0.3086719	total: 9m 52s	remaining: 1h 6m 27s
453:	learn: 0.3081253	total: 9m

579:	learn: 0.2732463	total: 12m 44s	remaining: 1h 4m 8s
580:	learn: 0.2731191	total: 12m 45s	remaining: 1h 4m 6s
581:	learn: 0.2728436	total: 12m 46s	remaining: 1h 4m 4s
582:	learn: 0.2724602	total: 12m 47s	remaining: 1h 4m 2s
583:	learn: 0.2721382	total: 12m 49s	remaining: 1h 4m
584:	learn: 0.2719116	total: 12m 50s	remaining: 1h 3m 59s
585:	learn: 0.2717228	total: 12m 52s	remaining: 1h 3m 59s
586:	learn: 0.2713888	total: 12m 53s	remaining: 1h 3m 56s
587:	learn: 0.2712284	total: 12m 54s	remaining: 1h 3m 55s
588:	learn: 0.2709301	total: 12m 55s	remaining: 1h 3m 52s
589:	learn: 0.2706867	total: 12m 56s	remaining: 1h 3m 50s
590:	learn: 0.2702239	total: 12m 58s	remaining: 1h 3m 49s
591:	learn: 0.2701049	total: 12m 59s	remaining: 1h 3m 48s
592:	learn: 0.2698196	total: 13m	remaining: 1h 3m 47s
593:	learn: 0.2695166	total: 13m 2s	remaining: 1h 3m 46s
594:	learn: 0.2692847	total: 13m 3s	remaining: 1h 3m 44s
595:	learn: 0.2691157	total: 13m 4s	remaining: 1h 3m 43s
596:	learn: 0.2687688	total: 

...

2897:	learn: 0.0590253	total: 1h 7m 30s	remaining: 14m 1s
2898:	learn: 0.0590005	total: 1h 7m 31s	remaining: 13m 59s
2899:	learn: 0.0589633	total: 1h 7m 32s	remaining: 13m 58s
2900:	learn: 0.0589528	total: 1h 7m 34s	remaining: 13m 57s
2901:	learn: 0.0589346	total: 1h 7m 35s	remaining: 13m 55s
2902:	learn: 0.0588971	total: 1h 7m 37s	remaining: 13m 54s
2903:	learn: 0.0588556	total: 1h 7m 38s	remaining: 13m 52s
2904:	learn: 0.0588375	total: 1h 7m 40s	remaining: 13m 51s
2905:	learn: 0.0587998	total: 1h 7m 41s	remaining: 13m 50s
2906:	learn: 0.0587878	total: 1h 7m 42s	remaining: 13m 48s
2907:	learn: 0.0587309	total: 1h 7m 43s	remaining: 13m 47s
2908:	learn: 0.0586555	total: 1h 7m 45s	remaining: 13m 45s
2909:	learn: 0.0586164	total: 1h 7m 47s	remaining: 13m 44s
2910:	learn: 0.0585887	total: 1h 7m 48s	remaining: 13m 43s
2911:	learn: 0.0585415	total: 1h 7m 50s	remaining: 13m 41s
2912:	learn: 0.0584967	total: 1h 7m 51s	remaining: 13m 40s
2913:	learn: 0.0584836	total: 1h 7m 53s	remaining: 13m 39

3037:	learn: 0.0550826	total: 1h 11m 5s	remaining: 10m 48s
3038:	learn: 0.0550224	total: 1h 11m 7s	remaining: 10m 47s
3039:	learn: 0.0549818	total: 1h 11m 8s	remaining: 10m 45s
3040:	learn: 0.0549432	total: 1h 11m 10s	remaining: 10m 44s
3041:	learn: 0.0549240	total: 1h 11m 12s	remaining: 10m 43s
3042:	learn: 0.0548717	total: 1h 11m 13s	remaining: 10m 41s
3043:	learn: 0.0548550	total: 1h 11m 15s	remaining: 10m 40s
3044:	learn: 0.0548320	total: 1h 11m 17s	remaining: 10m 39s
3045:	learn: 0.0547824	total: 1h 11m 18s	remaining: 10m 37s
3046:	learn: 0.0547607	total: 1h 11m 19s	remaining: 10m 36s
3047:	learn: 0.0547474	total: 1h 11m 21s	remaining: 10m 34s
3048:	learn: 0.0547224	total: 1h 11m 22s	remaining: 10m 33s
3049:	learn: 0.0546730	total: 1h 11m 24s	remaining: 10m 32s
3050:	learn: 0.0546446	total: 1h 11m 25s	remaining: 10m 30s
3051:	learn: 0.0546234	total: 1h 11m 27s	remaining: 10m 29s
3052:	learn: 0.0546139	total: 1h 11m 28s	remaining: 10m 27s
3053:	learn: 0.0545940	total: 1h 11m 30s	re

3177:	learn: 0.0515414	total: 1h 14m 35s	remaining: 7m 33s
3178:	learn: 0.0515240	total: 1h 14m 37s	remaining: 7m 32s
3179:	learn: 0.0515190	total: 1h 14m 38s	remaining: 7m 30s
3180:	learn: 0.0514877	total: 1h 14m 40s	remaining: 7m 29s
3181:	learn: 0.0514545	total: 1h 14m 41s	remaining: 7m 27s
3182:	learn: 0.0514284	total: 1h 14m 43s	remaining: 7m 26s
3183:	learn: 0.0513961	total: 1h 14m 44s	remaining: 7m 25s
3184:	learn: 0.0513823	total: 1h 14m 45s	remaining: 7m 23s
3185:	learn: 0.0513654	total: 1h 14m 47s	remaining: 7m 22s
3186:	learn: 0.0513442	total: 1h 14m 48s	remaining: 7m 20s
3187:	learn: 0.0512967	total: 1h 14m 50s	remaining: 7m 19s
3188:	learn: 0.0512696	total: 1h 14m 51s	remaining: 7m 18s
3189:	learn: 0.0512517	total: 1h 14m 53s	remaining: 7m 16s
3190:	learn: 0.0512378	total: 1h 14m 54s	remaining: 7m 15s
3191:	learn: 0.0512195	total: 1h 14m 55s	remaining: 7m 13s
3192:	learn: 0.0512060	total: 1h 14m 57s	remaining: 7m 12s
3193:	learn: 0.0511980	total: 1h 14m 59s	remaining: 7m 1

3317:	learn: 0.0484218	total: 1h 17m 58s	remaining: 4m 16s
3318:	learn: 0.0484014	total: 1h 17m 59s	remaining: 4m 15s
3319:	learn: 0.0483653	total: 1h 18m 1s	remaining: 4m 13s
3320:	learn: 0.0483422	total: 1h 18m 2s	remaining: 4m 12s
3321:	learn: 0.0483340	total: 1h 18m 3s	remaining: 4m 10s
3322:	learn: 0.0483267	total: 1h 18m 4s	remaining: 4m 9s
3323:	learn: 0.0482881	total: 1h 18m 6s	remaining: 4m 8s
3324:	learn: 0.0482578	total: 1h 18m 8s	remaining: 4m 6s
3325:	learn: 0.0482106	total: 1h 18m 9s	remaining: 4m 5s
3326:	learn: 0.0481949	total: 1h 18m 11s	remaining: 4m 3s
3327:	learn: 0.0481746	total: 1h 18m 12s	remaining: 4m 2s
3328:	learn: 0.0481548	total: 1h 18m 14s	remaining: 4m 1s
3329:	learn: 0.0481319	total: 1h 18m 15s	remaining: 3m 59s
3330:	learn: 0.0481059	total: 1h 18m 17s	remaining: 3m 58s
3331:	learn: 0.0480756	total: 1h 18m 18s	remaining: 3m 56s
3332:	learn: 0.0480342	total: 1h 18m 20s	remaining: 3m 55s
3333:	learn: 0.0479962	total: 1h 18m 22s	remaining: 3m 54s
3334:	learn

3458:	learn: 0.0454161	total: 1h 21m 31s	remaining: 58s
3459:	learn: 0.0453884	total: 1h 21m 33s	remaining: 56.6s
3460:	learn: 0.0453791	total: 1h 21m 35s	remaining: 55.2s
3461:	learn: 0.0453503	total: 1h 21m 36s	remaining: 53.8s
3462:	learn: 0.0453274	total: 1h 21m 38s	remaining: 52.3s
3463:	learn: 0.0452991	total: 1h 21m 39s	remaining: 50.9s
3464:	learn: 0.0452741	total: 1h 21m 41s	remaining: 49.5s
3465:	learn: 0.0452470	total: 1h 21m 43s	remaining: 48.1s
3466:	learn: 0.0452195	total: 1h 21m 44s	remaining: 46.7s
3467:	learn: 0.0452028	total: 1h 21m 46s	remaining: 45.3s
3468:	learn: 0.0451943	total: 1h 21m 47s	remaining: 43.9s
3469:	learn: 0.0451612	total: 1h 21m 49s	remaining: 42.4s
3470:	learn: 0.0451511	total: 1h 21m 50s	remaining: 41s
3471:	learn: 0.0451129	total: 1h 21m 52s	remaining: 39.6s
3472:	learn: 0.0450949	total: 1h 21m 53s	remaining: 38.2s
3473:	learn: 0.0450678	total: 1h 21m 55s	remaining: 36.8s
3474:	learn: 0.0450517	total: 1h 21m 56s	remaining: 35.4s
3475:	learn: 0.045

In [212]:
matrix(x_train_2,y_train_2,x_test_2,y_test_2,Cat_8127_6)


 Train accuracy 0.905

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.89      0.97      0.92     22535
functional needs repair       0.83      0.53      0.65      3024
         non functional       0.95      0.89      0.92     16021

               accuracy                           0.90     41580
              macro avg       0.89      0.80      0.83     41580
           weighted avg       0.90      0.90      0.90     41580


 Test accuracy 0.809

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.81      0.90      0.85      9724
functional needs repair       0.60      0.31      0.41      1293
         non functional       0.84      0.78      0.81      6803

               accuracy                           0.81     17820
              macro avg       0.75      0.66      0.69     17820
           weighted avg       0.80      0.

In [213]:
y_pred_9=Cat_8127_6.predict(Target1)
y_pred_9=pd.DataFrame(y_pred_9, columns=["status_group"])
y_pred_9["id"]=Target_x["id"]
y_pred_9=y_pred_9.reindex(["id","status_group"],axis=1)
y_pred_9.to_csv("F:/Personal/Queens MMAI/MMAI 869 – Machine Learning and AI Technology/Driven Data team Assignment/v6_out_cat_24.11_iter_3500_multiclass.csv",index=False)